<a href="https://colab.research.google.com/github/Akkki28/SparseMax-Transformers/blob/main/Decoder_Block.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch import nn
import matplotlib.pyplot as plt
import math
import seaborn as sns
import torch.nn.functional as F
from torch.autograd import Function
#import MultiHeadAttention

In [8]:
class DecoderBlock(nn.Module):

    def __init__(self, input_dim, num_heads, dim_feedforward, dropout=0.0):
        super().__init__()
        self.self_attn = MultiheadAttention(input_dim, input_dim, num_heads)

        self.cross_attn = MultiheadAttention(input_dim, input_dim, num_heads)

        self.linear_net = nn.Sequential(
            nn.Linear(input_dim, dim_feedforward),
            nn.Dropout(dropout),
            nn.ReLU(inplace=True),
            nn.Linear(dim_feedforward, input_dim)
        )

        self.norm1 = nn.LayerNorm(input_dim)
        self.norm2 = nn.LayerNorm(input_dim)
        self.norm3 = nn.LayerNorm(input_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, encoder_output, src_mask=None, tgt_mask=None):
        attn_out1 = self.self_attn(x, mask=tgt_mask)
        x = x + self.dropout(attn_out1)
        x = self.norm1(x)

        attn_out2 = self.cross_attn(x, encoder_output, mask=src_mask)
        x = x + self.dropout(attn_out2)
        x = self.norm2(x)

        linear_out = self.linear_net(x)
        x = x + self.dropout(linear_out)
        x = self.norm3(x)

        return x
